Иван Нужденко, группа c4111

Зависимости: python 3.5, anaconda3, plotly

In [74]:
import pandas as pd

Функция into_dataframe парсит приложенные текстовые файлы и конвертирует их в pd.DataFrame

In [75]:
def into_dataframe(filename):
    lst = []
    with open(filename, 'r') as f:
        for line in f:
            if not line.strip():
                break
            X, y = line.strip().rsplit(',', 1)
            lst.append([X, y])
    
    df = pd.DataFrame(lst,  columns=['message', 'label' ])
    df['target'] = df.label.map({'ham':0, 'spam':1})
    return df

In [76]:
df = into_dataframe('english_big.txt')

In [77]:
df.head(10)

,message,label,target
0,Urgent! call 09061749602 from Landline. Your c...,spam,1
1,+449071512431 URGENT! This is the 2nd attempt ...,spam,1
2,FREE for 1st week! No1 Nokia tone 4 ur mob eve...,spam,1
3,Urgent! call 09066612661 from landline. Your c...,spam,1
4,WINNER!! As a valued network customer you have...,spam,1
5,okmail: Dear Dave this is your final notice to...,spam,1
6,07732584351 - Rodger Burns - MSG = We tried to...,spam,1
7,"""URGENT! This is the 2nd attempt to contact U!...",spam,1
8,Congrats! Nokia 3650 video camera phone is you...,spam,1
9,Urgent! Please call 0906346330. Your ABTA comp...,spam,1


In [78]:
X = df.message
y = df.target

In [79]:
from collections import Counter

Counter(y)

Counter({0: 1002, 1: 322})

Разобьем выборку на тренировочную и тестовую в соотношении 80/20

In [80]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [81]:
X_train.shape, X_test.shape

((1059,), (265,))

Векторизуем sms-сообщения с помощью CountVectorizer

In [82]:
from sklearn.feature_extraction.text import CountVectorizer

In [83]:
vect = CountVectorizer()

X_train_sparse = vect.fit_transform(X_train)
X_train_sparse

<1059x3124 sparse matrix of type '<class 'numpy.int64'>'
	with 14366 stored elements in Compressed Sparse Row format>

In [84]:
X_train_human_readable = pd.DataFrame(X_train_sparse.toarray(), columns=vect.get_feature_names())

In [85]:
X_train_human_readable.head()

,00,000,000pes,008704050406,0121,01223585334,0125698789,02,021,03,...,ј350,ј400,ј450,ј48,ј500,ј5000,ј75,ј750,ј800,ј900
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
X_test_sparse = vect.transform(X_test)

In [87]:
X_test_sparse

<265x3124 sparse matrix of type '<class 'numpy.int64'>'
	with 2914 stored elements in Compressed Sparse Row format>

Обучим Байесовский классификатор и оценим качество модели

In [88]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, classification_report


In [89]:
multi = MultinomialNB()
multi.fit(X_train_sparse, y_train)
pred = multi.predict(X_test_sparse)
print(classification_report(pred, y_test))


             precision    recall  f1-score   support

          0       0.99      0.99      0.99       199
          1       0.98      0.98      0.98        66

avg / total       0.99      0.99      0.99       265



In [90]:
gauss = GaussianNB()
gauss.fit(X_train_sparse.toarray(), y_train)
pred_gauss = gauss.predict(X_test_sparse.toarray())
print(classification_report(pred_gauss, y_test))

             precision    recall  f1-score   support

          0       0.91      0.99      0.95       182
          1       0.98      0.78      0.87        83

avg / total       0.93      0.93      0.93       265



In [91]:
bernoulli = BernoulliNB()
bernoulli.fit(X_train_sparse.toarray(), y_train)
pred_bernoulli = bernoulli.predict(X_test_sparse.toarray())
print(classification_report(pred_bernoulli, y_test))

             precision    recall  f1-score   support

          0       1.00      0.99      0.99       201
          1       0.97      1.00      0.98        64

avg / total       0.99      0.99      0.99       265



Без настройки параметров и кросс-валидации лучший результат показывает MultinomialNB и BernoulliNB

In [92]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.graph_objs as go

init_notebook_mode(connected=True)

Посчитаем какие слова наиболее часто появлялись в сообщениях, помеченных как "спам" и как "не спам". Будем использовать результаты алгоритма MultinomialNB

Метод .feature_count_ показывает сколько раз каждое слово из набора было в сообщении классифицированном, как спам

In [93]:
X_train_tokens = vect.get_feature_names()
multi.feature_count_

array([[  0.,   0.,   1., ...,   0.,   0.,   0.],
       [  5.,  14.,   0., ...,   1.,   5.,   7.]])

In [94]:
ham_token_count = multi.feature_count_[0, :]
spam_token_count = multi.feature_count_[1, :]

In [95]:
tokens = pd.DataFrame({'token':X_train_tokens, 'ham':ham_token_count+1, 'spam':spam_token_count+1}).set_index('token')
tokens.head()

,ham,spam
token,,
00,1.0,6.0
000,1.0,15.0
000pes,2.0,1.0
008704050406,1.0,2.0
0121,1.0,2.0


То же самое в относительных показателях

In [96]:
tokens['ham_relative'] = tokens.ham / multi.class_count_[0]
tokens['spam_relative'] = tokens.spam / multi.class_count_[1]
tokens['spam_ratio'] = tokens.spam_relative / tokens.ham_relative
tokens.head()

,ham,spam,ham_relative,spam_relative,spam_ratio
token,,,,,
00,1.0,6.0,0.001245,0.023438,18.820312
000,1.0,15.0,0.001245,0.058594,47.050781
000pes,2.0,1.0,0.002491,0.003906,1.568359
008704050406,1.0,2.0,0.001245,0.007812,6.273438
0121,1.0,2.0,0.001245,0.007812,6.273438


Топ-20 слов, появляющихся в сообщениях с пометкой "спам"

In [97]:
tokens.sort_values('spam_ratio', ascending=False).head(20)

,ham,spam,ham_relative,spam_relative,spam_ratio
token,,,,,
prize,1.0,53.0,0.001245,0.207031,166.246094
claim,1.0,53.0,0.001245,0.207031,166.246094
www,1.0,34.0,0.001245,0.132812,106.648438
urgent,1.0,34.0,0.001245,0.132812,106.648438
contact,1.0,33.0,0.001245,0.128906,103.511719
150p,1.0,32.0,0.001245,0.125000,100.375000
nokia,1.0,31.0,0.001245,0.121094,97.238281
cash,1.0,30.0,0.001245,0.117188,94.101562
win,1.0,27.0,0.001245,0.105469,84.691406


Топ-20 слов, появляющихся в сообщениях с пометкой "не спам"

In [98]:
tokens.sort_values('spam_ratio', ascending=True).head(20)

,ham,spam,ham_relative,spam_relative,spam_ratio
token,,,,,
lor,127.0,1.0,0.158157,0.003906,0.024699
wat,74.0,1.0,0.092154,0.003906,0.042388
da,52.0,1.0,0.064757,0.003906,0.060322
already,48.0,1.0,0.059776,0.003906,0.065348
dun,47.0,1.0,0.058531,0.003906,0.066739
but,92.0,2.0,0.114570,0.007812,0.068190
home,46.0,1.0,0.057285,0.003906,0.068190
thk,44.0,1.0,0.054795,0.003906,0.071289
come,43.0,1.0,0.053549,0.003906,0.072947


Потюнингуем параметры алгоритмов, обработчика текста, запустим кросс-валидацию и снова сравним модели

Для обработчика CountVectorizer добавлен стандартный список стоп-слов, добавлены N-грамы длины 2 и 3. Также отсекаются слова, которые попадаются более, чем в половине документов 

In [99]:
tuned_vect = CountVectorizer(stop_words='english', ngram_range=(1, 3), max_df=0.5)

In [100]:
df = into_dataframe('english_big.txt')
X = df.message
y = df.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [101]:
X_train_sparse = tuned_vect.fit_transform(X_train)
X_train_sparse

<1059x16937 sparse matrix of type '<class 'numpy.int64'>'
	with 26520 stored elements in Compressed Sparse Row format>

In [102]:
X_test_sparse = tuned_vect.transform(X_test)

In [103]:
multi = MultinomialNB(alpha=.1)
multi.fit(X_train_sparse, y_train)
pred = multi.predict(X_test_sparse)
print(classification_report(pred, y_test))

             precision    recall  f1-score   support

          0       0.99      0.99      0.99       199
          1       0.98      0.98      0.98        66

avg / total       0.99      0.99      0.99       265



In [104]:
gauss = GaussianNB()
gauss.fit(X_train_sparse.toarray(), y_train)
pred_gauss = gauss.predict(X_test_sparse.toarray())
print(classification_report(pred_gauss, y_test))

             precision    recall  f1-score   support

          0       0.90      0.99      0.94       180
          1       0.98      0.76      0.86        85

avg / total       0.93      0.92      0.92       265



In [105]:
bernoulli = BernoulliNB(.1)
bernoulli.fit(X_train_sparse.toarray(), y_train)
pred_bernoulli = bernoulli.predict(X_test_sparse.toarray())
print(classification_report(pred_bernoulli, y_test))

             precision    recall  f1-score   support

          0       1.00      0.97      0.98       206
          1       0.89      1.00      0.94        59

avg / total       0.98      0.97      0.97       265



Запустим кросс-валидационную сетку и оценим получившиеся результаты

In [106]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [107]:
BernoulliNB().get_params().keys()

dict_keys(['class_prior', 'binarize', 'fit_prior', 'alpha'])

In [108]:
params = {'bernoulli__alpha': list(i / 10 for i in range(1, 11))}
pipe = Pipeline([('bernoulli', BernoulliNB())])
bernoulli_grid = GridSearchCV(pipe, params,
                        cv=5, n_jobs=-1,
                        verbose=True)

In [109]:
bernoulli_grid.fit(X_train_sparse.toarray(), y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.6min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('bernoulli', BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'bernoulli__alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=True)

In [110]:
bernoulli_grid.best_params_, bernoulli_grid.best_score_

({'bernoulli__alpha': 0.1}, 0.97639282341831912)

In [111]:
results = bernoulli_grid.cv_results_['mean_test_score']
legend = list(i / 10 for i in range(1, 11))

trace1 = go.Scatter(
        x=legend,
        y=results,
        showlegend=False)



data = go.Data([trace1])

layout=go.Layout(
        yaxis=go.YAxis(title='Accuracy score'),
        xaxis=go.XAxis(title='Aplha parameter value', dtick=0.1,),
        title='Bernoulli classifier prediction on training set with different alpha parameter values')

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [112]:
pred_bernoulli_grid = bernoulli_grid.predict(X_test_sparse.toarray())

In [113]:
print(classification_report(pred_bernoulli_grid, y_test))

             precision    recall  f1-score   support

          0       1.00      0.97      0.98       206
          1       0.89      1.00      0.94        59

avg / total       0.98      0.97      0.97       265



Оттюнингованная сетка дала точность предсказания в 97%. Точность прогноза на тренировочной выборке также составляет 97%. 

Попробуем тот же метод на алгоритме MultinomialNB

In [114]:
params = {'multi__alpha': list(i / 10 for i in range(1, 11))}
pipe = Pipeline([('multi', MultinomialNB())])
multi_grid = GridSearchCV(pipe, params,
                        cv=5, n_jobs=-1,
                        verbose=True)

In [115]:
multi_grid.fit(X_train_sparse.toarray(), y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.2min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('multi', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'multi__alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=True)

In [116]:
multi_grid.best_params_, multi_grid.best_score_

({'multi__alpha': 0.8}, 0.99055712936732765)

In [117]:
multi_res = multi_grid.cv_results_['mean_test_score']
bernoulli_res = bernoulli_grid.cv_results_['mean_test_score']
legend = list(i / 10 for i in range(1, 11))

trace1 = go.Scatter(
        x=legend,
        y=bernoulli_res,
        showlegend=True, 
        name='BernoulliNB')
trace2 = go.Scatter(
        x=legend, 
        y=multi_res, 
        showlegend=True, 
        name='MultinomialNB')


data = go.Data([trace1, trace2])

layout=go.Layout(
        yaxis=go.YAxis(title='Accuracy score'),
        xaxis=go.XAxis(title='Aplha parameter value', dtick=0.1,),
        title='BernoulliNB vs MultinomialNB classifier prediction on training set with different alpha parameter values')

fig = go.Figure(data=data, layout=layout)
iplot(fig)

Алгоритм MultinomialNB показывает более высокие предсказательные результаты на тренировочной выборке. Предсказания, сделанные на тестовой выборке у него также точнее (99% vs 97%)

In [118]:
pred_multi_grid = multi_grid.predict(X_test_sparse.toarray())
print(classification_report(pred_multi_grid, y_test))

             precision    recall  f1-score   support

          0       0.99      0.99      0.99       199
          1       0.98      0.98      0.98        66

avg / total       0.99      0.99      0.99       265



In [119]:
gauss = GaussianNB()
gauss.fit(X_train_sparse.toarray(), y_train)
pred_gauss = gauss.predict(X_test_sparse.toarray())
print(classification_report(pred_gauss, y_test))

             precision    recall  f1-score   support

          0       0.90      0.99      0.94       180
          1       0.98      0.76      0.86        85

avg / total       0.93      0.92      0.92       265




Алгоритм GaussianNB показывает точность 92% и проигрывает двум предыдущим


In [120]:
accuracies_with_cv = [.99, 0.97, 0.92]
legend = ['MultinomialNB','BernoulliNB', 'GaussianNB']

trace1 = go.Scatter(
        x=legend,
        y=accuracies_with_cv,
        showlegend=False)



data = go.Data([trace1])

layout=go.Layout(
        yaxis=go.YAxis(title='Accuracy score'),
        title='Prediction quality with cross-validation and model tuning ')

fig = go.Figure(data=data, layout=layout)
iplot(fig)

Поскольку в этой части работы был по другому настроен обработчик текста, имеет смысл снова проверить какие слова алгоритм отмечал как наиболее часто маркирующие спам.

In [121]:
X_train_tokens = tuned_vect.get_feature_names()
multi.feature_count_

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 5.,  1.,  1., ...,  2.,  1.,  1.]])

In [122]:
ham_token_count = multi.feature_count_[0, :]
spam_token_count = multi.feature_count_[1, :]

In [123]:
tokens = pd.DataFrame({'token':X_train_tokens, 'ham':ham_token_count+1, 'spam':spam_token_count+1}).set_index('token')
tokens.head()

,ham,spam
token,,
00,1.0,6.0
00 easter,1.0,2.0
00 easter prize,1.0,2.0
00 sub,1.0,3.0
00 sub 16,1.0,3.0


In [124]:
tokens['ham_relative'] = tokens.ham / multi.class_count_[0]
tokens['spam_relative'] = tokens.spam / multi.class_count_[1]
tokens['spam_ratio'] = tokens.spam_relative / tokens.ham_relative
tokens.head()

,ham,spam,ham_relative,spam_relative,spam_ratio
token,,,,,
00,1.0,6.0,0.001245,0.023438,18.820312
00 easter,1.0,2.0,0.001245,0.007812,6.273438
00 easter prize,1.0,2.0,0.001245,0.007812,6.273438
00 sub,1.0,3.0,0.001245,0.011719,9.410156
00 sub 16,1.0,3.0,0.001245,0.011719,9.410156


Слова, чаще всего попадающиеся в сообщениях, помеченных как "спам"

In [125]:
tokens.sort_values('spam_ratio', ascending=False).head(20)

,ham,spam,ham_relative,spam_relative,spam_ratio
token,,,,,
claim,1.0,53.0,0.001245,0.207031,166.246094
prize,1.0,53.0,0.001245,0.207031,166.246094
urgent,1.0,34.0,0.001245,0.132812,106.648438
www,1.0,34.0,0.001245,0.132812,106.648438
contact,1.0,33.0,0.001245,0.128906,103.511719
150p,1.0,32.0,0.001245,0.125000,100.375000
nokia,1.0,31.0,0.001245,0.121094,97.238281
cash,1.0,30.0,0.001245,0.117188,94.101562
guaranteed,1.0,27.0,0.001245,0.105469,84.691406


Слова, чаще всего попадающиеся в сообщениях, помеченных как "не спам"

In [126]:
tokens.sort_values('spam_ratio', ascending=True).head(20)

,ham,spam,ham_relative,spam_relative,spam_ratio
token,,,,,
lor,127.0,1.0,0.158157,0.003906,0.024699
wat,74.0,1.0,0.092154,0.003906,0.042388
da,52.0,1.0,0.064757,0.003906,0.060322
dun,47.0,1.0,0.058531,0.003906,0.066739
home,46.0,1.0,0.057285,0.003906,0.068190
thk,44.0,1.0,0.054795,0.003906,0.071289
come,43.0,1.0,0.053549,0.003906,0.072947
hey,41.0,1.0,0.051059,0.003906,0.076505
im,40.0,1.0,0.049813,0.003906,0.078418
